<a href="https://colab.research.google.com/github/yttinalee/data-course-sample/blob/main/Week3_A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [ ]:
metadata = getDF('../S2/meta_All_Beauty.json.gz')
ratings = pd.read_csv('http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_trainings_by_user = ratings_trainings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_trainings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_trainings_by_user }
training_users = list(ratings_trainings_by_user.keys())
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
same_user = set(training_users)&set(users)

In [ ]:
notsame_user = set(users)^set(same_user)

In [ ]:
len(notsame_user), len(users)

(546, 584)

In [ ]:
ratings_trainings_by_asin = ratings_testings.groupby('asin').agg(list).reset_index()['asin']
# ratings_trainings_by_asin = { rating['reviewerID']: rating['asin'] for rating in ratings_trainings_by_user }
# training_asins = list(ratings_trainings_by_user.keys())
ratings_testings_by_asin = ratings_testings.groupby('asin').agg(list).reset_index()['asin']
# ratings_testings_by_asin = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
# asins = list(ratings_testings_by_user.keys())

In [ ]:
# # training data中的asin有多少也出現在testint data
same_asin = set(ratings_trainings_by_asin)&set(ratings_testings_by_asin)
len(same_asin)

335

In [ ]:
category = metadata['rank'].str.split(' in ', expand=True)
category = category.rename(columns={category.columns[0]:'ranking', category.columns[1]:'categories'})
# category.head()

In [ ]:
category['ranking'] = category['ranking'].str.replace(',','')
category['categories'] = category['categories'].str.replace('amp;','')
category['categories'] = category['categories'].str.replace('\(', '')
category['ranking'] = pd.to_numeric(category['ranking'])

In [ ]:
meta_use = metadata[['asin', 'title', 'also_buy', 'brand', 'rank', 'also_view', 'details', 'price', 'description']]

In [ ]:
meta_use.groupby('asin').count()['title'].sort_values(ascending=False)

asin
B00026KYLM    2
B00021DI10    2
B00021CWYO    2
B00021CQTA    2
B00021CPNC    2
             ..
B01172CGQK    1
B01172N21I    1
B01172N2HM    1
B01174YHM4    1
6546546450    1
Name: title, Length: 32488, dtype: int64

In [ ]:
meta_use.groupby('brand').count()['title'].sort_values(ascending=False).tail(50)

brand
NeuOra Microceuticals                                       1
Neugaugh                                                    1
Nippon Kodo                                                 1
Njoy, Wicked                                                1
No Boundries                                                1
No Nix                                                      1
No Turn                                                     1
NoCry                                                       1
Noix D'Or                                                   1
Nok Out                                                     1
Nomaterra                                                   1
Noogleberry                                                 1
Norelco 6701X Cool Skin Additive In/Out of Shower Shaver    1
Noreva                                                      1
North American Hemp Company                                 1
North American Herb & Spice                                 1
No

In [ ]:
meta_use.groupby('title').count()['asin'].sort_values(ascending=False).head(10)

title
Baubax Women's Bomber Travel Jacket                                                                                                                                                          9
Bourjois Healthy Mix Foundation                                                                                                                                                              7
Derma Roller 0.25mm - Facial Skin Care Tool for Face - Micro Needle Roller - 540 Deluxe Titanium Micro Needles - Titanium Microneedle Roller - Perfect Cosmetic Microneedling Skin Roller    7
Sankuwen 15PCs Wool Makeup Brush Set Tools Toiletry Kit                                                                                                                                      7
2                                                                                                                                                                                            5
BMC Nail Art Water Transfer Tattoo Effe

In [ ]:
meta_use['ranking'] = category['ranking']
meta_use['categories'] = category['categories']
meta_use['title'] = meta_use['title'].str.replace('&amp;','')

<ipython-input-196-4095d30288ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['ranking'] = category['ranking']
<ipython-input-196-4095d30288ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['categories'] = category['categories']
<ipython-input-196-4095d30288ae>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [ ]:
len(meta_use['title'].unique()), len(meta_use['title']), meta_use['title'].head(7)

(32300,
 32892,
 0        Loud 'N Clear&trade; Personal Sound Amplifier
 1    No7 Lift  Luminate Triple Action Serum 50ml by...
 2      No7 Stay Perfect Foundation Cool Vanilla by No7
 3    Wella Koleston Perfect Hair Colour 44/44 Mediu...
 4    Lacto Calamine Skin Balance Oil control 120 ml...
 5    Mary Kay Satin Hands Hand Cream Travel MINI Si...
 6    Unique Custom Cast Iron Liner Shader Tattoo Ma...
 Name: title, dtype: object)

In [ ]:
len(ratings_trainings['reviewerID'].unique()), len(ratings_trainings['reviewerID']), len(ratings_trainings.groupby(['reviewerID']).count()>1)

(323489, 370752, 323489)

In [ ]:
training_users_2uper_uni = ratings_trainings.groupby(['reviewerID']).count()['asin'][ratings_trainings.groupby(['reviewerID']).count()['asin']>1]

In [ ]:
a = []
for ID in range(0, len(ratings_trainings['reviewerID'])):
  if ratings_trainings['reviewerID'].iloc[ID] in training_users_2uper_uni:
    a.append(ID)
ratings_trainings_remove1 = ratings_trainings.iloc[a]

In [ ]:
len(ratings_trainings_remove1['reviewerID'].unique()), len(ratings_trainings_remove1)

(36222, 83485)

In [ ]:
ratings_users_uni = ratings.groupby(['reviewerID']).count()['asin'][ratings.groupby(['reviewerID']).count()['asin']>1]
a = []
for ID in range(0, len(ratings['reviewerID'])):
  if ratings['reviewerID'].iloc[ID] in ratings_users_uni:
    a.append(ID)
ratings_trainings_remove1_surprise= ratings.iloc[a]

In [ ]:
len(ratings_trainings_remove1_surprise)

83561

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

lemmatizer = WordNetLemmatizer()

df = meta_use.drop_duplicates(['asin', 'title', 'brand'])

def preproc(sentence):
# for sentence in range(0, len(meta_use['title'])):
  remove_punctuation = ''.join([i for i in sentence if i not in string.punctuation]).lower()
  tokens = word_tokenize(remove_punctuation)
  result = [lemmatizer.lemmatize(word) for word in tokens if not word in stopwords.words('english')]
  return ' '.join(result)

df['title'].apply(lambda x: preproc(x))
# 計算商品用標題所表示的 tfidf 矩陣

tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

# 計算商品間的相似程度
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# # 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# # 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
limitTime_training_data = ratings_trainings[(ratings_trainings['DATE'] >= '2018-05-31') &  (ratings_trainings['DATE'] <= '2018-09-01')]
top_10 = limitTime_training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]

In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def recommender_user(training_data, users=[], k=10):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    # print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    # print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    # print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    # print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
    return recommendation 

def recommender_item(training_data, users=[], k=10):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    # print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    # print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    # print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    # print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
    return recommendation

import time
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender_surprise(training_data, users=[], k=10, user_based=False, algo=KNNBasic):

    training_data = training_data[(training_data['DATE'] >= '2016-08-31') &  (training_data['DATE'] <= '2018-09-30')]

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list

    return recommendation     

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score
# evaluate(ratings_testings_by_user, ratings_by_user)

In [ ]:
## Test only user-based or item-based results
evaluate_result = {}
n = 0
R = [recommender_user, recommender_item]  
for recommender in R: 
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings, users)
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result


,recommender_list,score
0,0,0.001695
1,1,0.001695


In [ ]:
## Test user-based or item-based results with ID counts more than 1
evaluate_result = {}
n = 0
R = [recommender_user, recommender_item]  
for recommender in R: #
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings_remove1_surprise, users)  ## using removed only 1 history record ID
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result

,recommender_list,score
0,0,0.0
1,1,0.0


In [ ]:
evaluate_result = {}
n = 0
R = [recommender_surprise]  
train_data = [ratings, ratings_trainings_remove1_surprise]
for recommender in R: #
  for data in train_data:
    ratings_by_user = {}
    # print(ratings_by_user)
    ratings_by_user = recommender(data, users)  
    evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
    n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


,recommender_list,score
0,0,0.000000
1,1,0.001695


In [ ]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def recommender_user(training_data, users=[], k=10):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    # print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    # print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    # print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    # print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
  
        if len(recommendation[user]) < 10:
          recommend_cates = []
          recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k)
          [recommendation[user].append(recommend_cates[gg]) for gg in range(0, len(recommend_cates))]
          [recommendation[user].append(top_10[gg]) for gg in range(0, 10-len(recommendation[user]))]
    return recommendation    

def recommender_item(training_data, users=[], k=10):

    # loading data from dataframe
    # item_to_users dict:
    # {
    #   'item': {
    #       'user': ratings...
    #   }...
    # }
    item_to_users = defaultdict(dict)
    for _, row in training_data.iterrows():
        row = dict(row)
        user = row['reviewerID']
        item = row['asin']
        rating = float(row['overall'])
        item_to_users[item][user] = rating

    # print("data converted")

    user_to_items = defaultdict(dict)
    for item, rating_users in item_to_users.items():
        for user, rating in rating_users.items():
            user_to_items[user][item] = rating

    # print("data inverted")

    init_sim = lambda: [0, 0, 0]
    factory = lambda: defaultdict(init_sim)
    pre_item_similarity = defaultdict(factory)
    for user, items in user_to_items.items():
        if len(items) > 1:
            for i1, i2 in combinations(items.keys(), 2):
                xy = items[i1] * items[i2]
                xx = items[i1] ** 2
                yy = items[i2] ** 2
                pre_item_similarity[i1][i2][0] += xy
                pre_item_similarity[i1][i2][1] += xx
                pre_item_similarity[i1][i2][2] += yy

                pre_item_similarity[i2][i1][0] += xy
                pre_item_similarity[i2][i1][1] += xx
                pre_item_similarity[i2][i1][2] += yy

    # print("sim data prepared")

    item_similarity = {}
    for src_item in pre_item_similarity:
        item_similarity_order = []
        for dst_item, val in pre_item_similarity[src_item].items():
            xy = val[0]
            xx = val[1]
            yy = val[2]
            div = ((xx*yy) ** 0.5)
            if div == 0:
                continue
            similarity = xy / div
            if similarity < 0:
                continue
            for i, s in enumerate(item_similarity_order):
                target_similarity = s[1]
                if target_similarity < similarity:
                    item_similarity_order.insert(i, (dst_item, similarity))
                    break
            else:
                item_similarity_order.append((dst_item, similarity))
        item_similarity[src_item] = item_similarity_order

    # print(f"get {k} recommendation items for for user: {users}")

    recommendation = {}
    for user in users:
        items = []
        items_set = set()
        stop = False
        user_has_rated = set(user_to_items[user])
        for item in user_has_rated:
            if item in item_similarity:
                for sim_item, _ in item_similarity[item]:
                    # skip the item user has rated
                    if sim_item not in user_has_rated and sim_item not in items_set:
                        items.append(sim_item)
                        items_set.add(sim_item)
                    if len(items) >= k:
                        stop = True
                        break
                if stop:
                    break
        recommendation[user] = items
        if len(recommendation[user]) < 10:
          recommend_cates = []
          recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k)
          [recommendation[user].append(recommend_cates[gg]) for gg in range(0, len(recommend_cates))]
          [recommendation[user].append(top_10[gg]) for gg in range(0, 10-len(recommendation[user]))]
    return recommendation  

import time
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic

def recommender_surprise(training_data, users=[], k=10, user_based=False, algo=KNNBasic):

    training_data = training_data[(training_data['DATE'] >= '2016-08-31') &  (training_data['DATE'] <= '2018-09-01')]

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': 'cosine',
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list
        if len(recommendation[user]) < 10:
          recommend_cates = []
          recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k)
          [recommendation[user].append(recommend_cates[gg]) for gg in range(0, len(recommend_cates))]
          [recommendation[user].append(top_10[gg]) for gg in range(0, 10-len(recommendation[user]))]
    return recommendation

In [ ]:
evaluate_result = {}
n = 0
R = [recommender_user, recommender_item]  #recommender_user, recommender_item, recommender_surprise
for recommender in R: #
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings, users)
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result

,recommender_list,score
0,0,0.133898
1,1,0.133898


In [ ]:
evaluate_result = {}
n = 0
R = [recommender_user, recommender_item]  #recommender_user, recommender_item, recommender_surprise
for recommender in R: #
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings_remove1, users)
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result

,recommender_list,score
0,0,0.133898
1,1,0.133898


In [ ]:
evaluate_result = {}
n = 0
R = [recommender_surprise]  #recommender_user, recommender_item, recommender_surprise
train_data = [ratings, ratings_trainings_remove1_surprise]
for recommender in R: #
  for data in train_data:  
    ratings_by_user = {}
    # print(ratings_by_user)
    ratings_by_user = recommender(data, users)
    evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
    n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result